In [ ]:
import cv2
import numpy as np
from preprocessing import identify_face_etc as iden
from altres.colourspaces import hsl_calculations as hcalc, hsl_colourspace_stuff as hcol

Segment face and skin

In [ ]:
image_path = "../../data/images/spill01.jpg"
class_indices = [2, 3]
seg_img = iden.segment_face_et_al(og_img_path=image_path, class_indices=class_indices)
# NOTE: bin_mask and seg_img are numpy arrays

Find target skin tone

In [ ]:
hsv_seg = cv2.cvtColor(seg_img, cv2.COLOR_BGR2HSV)  # HSV
hls_seg = cv2.cvtColor(seg_img, cv2.COLOR_BGR2HLS)    # HLS
hue, lum, sat = cv2.split(hls_seg)  # Hue, Luminosity, Saturation

# Create skin mask
lower = np.array([0, 40, 20])           # Hue, Luminosity, Saturation
upper = np.array([80, 220, 225])     # Hue, Luminosity, Saturation
skin_mask = cv2.inRange(hls_seg, lower, upper)
skin_pixels = hls_seg[skin_mask > 0]


In [8]:
hsv_seg.shape

(472, 630, 3)

In [ ]:
# TODO: See gpt for binning clusters and getting hsl values not just hue  - hsl_calculations.py
# Get dominant hue
dom_hue = hcalc.get_dom_hue(skin_pix_mask=skin_pixels)


Detect green spill

In [ ]:
# Create heatmap
save_as = "outputs/og_heatmap.png"
hcol.create_colour_heatmap(hsv_img=hsv_seg, targ_hue=dom_hue, save_as=save_as)

# Create binary spill mask
hcol.create_bin_spill_mask(hsv_img=hsv_seg, targ_hue=dom_hue, display=True)


Change green spill to target skin hue

In [ ]:
corrected_image = hcol.replace_spill(hsv_img=hsv_seg, targ_hue=dom_hue, display=True)


Check it has improved the image

In [ ]:
corrected_image_hsv = cv2.cvtColor(corrected_image, cv2.COLOR_BGR2HSV)
save_as = "outputs/corrected_heatmap.png"
hcol.create_colour_heatmap(hsv_img=corrected_image_hsv, targ_hue=dom_hue, save_as=save_as)
